# Chapter 7. Tracking A 
---------------------------------------------


In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import numpy.linalg
from ipywidgets import *
from IPython.display import display
from IPython.core.display import HTML
from notebook.nbextensions import enable_nbextension 


def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()



### Comments

This chapter focuses on estimation.  An example is to estimate the location of an airplane given radar information and to update that estimate over time.  The mathematical formulation is least squares estimation. We start with the linear case.



## 7.1 Linear Regression

You measure the weight $Y_n$ and the height $X_n$ of person $n$ for $n = 1, \ldots, N$.  Your goal is to undertand the relationship between these two quantities, so that by observing the height $X$ of new individual you could estimate her weight $Y$.  Ideally, you may want to discover a function $g(\cdot)$ such that $Y = g(X).$  However, this is clearly not possible since many people with the same height have widely different weights.  Thus, the best one can hope for is a function $g(\cdot)$ such that $g(X)$ is a good guess for $Y$.  Mathematically, we look for a function $g(\cdot)$ that minimizes $E((Y - g(X))^2)$.  As a first step, we look for a linear function $g(X) = a + bY$.  Thus, the problem is to choose $a$ and $b$ to minimize

$$\frac{1}{N} \sum_{n=1}^N (Y_n - a - bX_n)^2.$$

The solution is

$$g(X) = E_N(Y) + \frac{cov_N(Y, X)}{var_N(X)} (X - E_N(X)).$$

In this expression,

\begin{eqnarray*}
E_N(X) &=& \frac{1}{N} \sum_{n=1}^N X_n \\
E_N(Y) &=& \frac{1}{N} \sum_{n=1}^N Y_n \\
cov_N(X,Y) &=& \frac{1}{N} \sum_{n=1}^N X_n Y_n - E_N(X)E_N(Y) \\
var_N(X) &=& \frac{1}{N} \sum_{n=1}^N X^2_n - E_N(X)^2.
\end{eqnarray*}

That is, $E_N(X)$ is the sample mean of $X$, i.e., the arithmetic mean of the samples $X_n$.  Similarly, $cov_N(X,Y)$ is the sample covariance, etc. 

### Examples

In the code below, we provide some examples.  In case (a), the samples are positively correlated.  In case (b), they are negatively correlated.

In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
import random
import numpy.linalg
from ipywidgets import *
from IPython.display import display



def LR(w,M):  # LR 
    N = M
    S = 100
    a = np.arange(0.0,N)
    b = np.arange(0.0,N)
    c = np.arange(0.0,S)
    d = np.arange(0.0,S)
    
    for k in range(N):
        a[k]= np.random.normal()
        if w == '(a)':
            b[k] =  2*a[k] +  1.5*np.random.normal()
        elif w == '(b)':
            b[k] =  - 2*a[k] +  np.random.normal()
    EX = 0
    EY = 0
    CXY = 0
    VX = 0
    for k in range(N):
        EX = EX + a[k]
        EY = EY + b[k]
        CXY = CXY + a[k]*b[k]
        VX = VX + a[k]**2
    EX = EX/N
    EY = EY/N
    CXY = CXY/N - EX*EY
    VX = VX/N - EX**2
    amin = min(a)
    amax = max(a)
    for s in range(S):
        c[s] = amin + s*(amax - amin)/S
        d[s] = EX + (CXY/VX)*(c[s]- EX)
    area = np.pi*3  
    plt.figure(figsize = (14,6))
    plt.scatter(a, b, s=area, c='b')
    plt.plot(c,d,c='r')
    plt.title('Pairs of random variables')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
            
u = widgets.ToggleButtons(options=['(a)', '(b)'],
    description='Case',
    disabled=False,
    button_style='info', 
    tooltip='Description',
#     icon='check'
) 



u2 = widgets.IntSlider(description='N', min = 10, max = 200, step = 10, value = 100, position = 'bottom')
        
z = widgets.interactive(LR, w = u, M = u2) 
display(z)
        

## 7.3 Quadratic Regression

In many situations, the best linear regression is not a good approximation.  For instance, if you want to estimate the area of an almost corcular object based on its diameter, the best guess is probably proportional to the square of the diameter.  Similarly, the salary of someone increases faster than linearly with the number of years of schooling. (There are exceptions, like me.)

The **Quadratic Least Squares Regression (QR)** of $Y$ over $X$ is the function $Y = a + bX + cX^2$
where $(a, b, c)$ minimize the sum $C$ of the squares of the prediction errors where

$$C = \sum_{m=1}^n (Y_m - a - bX_m - cX_m^2)^2.$$

As the book explains, $(a, b, c)$ are such that

\begin{eqnarray*}
&& \sum_{m=1}^n (Y_m - a - bX_m - cX_m^2) = 0 \\
&& \sum_{m=1}^n (Y_m - a - bX_m - cX_m^2)X_m = 0 \\
&& \sum_{m=1}^n (Y_m - a - bX_m - cX_m^2)X_m^2 = 0.
\end{eqnarray*}

Thus, $(a, b, c)$ solve the following linear system of equations:

$$ \left[
\begin{array}{c c c}
n & \sum_m X_m & \sum_m X_m^2 \\
\sum_m X_m & \sum_m X_m^2 & \sum_m X_m^3 \\
\sum_m X_m^2 &  \sum_m X_m^3 & \sum_m X_m^4 
\end{array}
\right]
\left[
\begin{array}{c}
a \\
b \\
c
\end{array}
\right]
=
\left[
\begin{array}{c}
\sum_m Y_m \\
\sum_m Y_mX_m \\
\sum_m Y_mX_m^2
\end{array}
\right].
$$
Thus, one can solve these equations and get the QLR.  

### Examples

We plote the QPR and LR on the data $Y_n = 0.2 - X_n + 2X_n^2 + 1.3Z_n$ in case (a) and $Y_n = 0.4 + X_n - 2 X_n^2 + 1.3Z_n$ in case (b) where $X_n$ and $Z_n$ are i.i.d. ${\cal N}(0, 1)$.  


In [ ]:
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
import random
import numpy.linalg
from ipywidgets import *
from IPython.display import display


def LR(w,M):  # Simulation of QR 
    N = M
    S = 100
    a = np.arange(0.0,N)
    b = np.arange(0.0,N)
    c = np.arange(0.0,S)
    d = np.arange(0.0,S)
    e = np.arange(0.0,S)
    
    for k in range(N):
        a[k]= np.random.normal()
        if w == '(a)':
            b[k] =  0.2 - a[k] +  2*a[k]**2 + 1.3*np.random.normal()
        elif w == '(b)':
            b[k] =  0.4 + a[k] -  2*a[k]**2 + 1.3*np.random.normal()
    Sx = 0
    Sx2 = 0
    Sx3 = 0
    Sx4 = 0
    Sy = 0
    Sxy = 0
    Sx2y = 0
    for k in range(N):
        Sx = Sx + a[k]
        Sx2 = Sx2 + a[k]**2
        Sx3 = Sx3 + a[k]**3
        Sx4 = Sx4 + a[k]**4
        Sy = Sy + b[k]
        Sxy = Sxy + a[k]*b[k]
        Sx2y = Sx2y + a[k]**2*b[k]
    A = [[N,Sx,Sx2],[Sx,Sx2,Sx3],[Sx2,Sx3,Sx4]]
    Ainv = np.linalg.inv(A)
    V = [Sy, Sxy, Sx2y]
    W = np.dot(Ainv,V)
    AL = [[N, Sx],[Sx, Sx2]]
    ALinv = np.linalg.inv(AL)
    VL = [Sy, Sxy]
    WL = np.dot(ALinv,VL)
    amin = min(a)
    amax = max(a)
    for s in range(S):
        c[s] = amin + s*(amax - amin)/S
        d[s] = W[0]+c[s]*W[1]+W[2]*c[s]**2
        e[s] = WL[0] + c[s]*WL[1]
    area = np.pi*3
    plt.figure(figsize = (14,6))
    plt.scatter(a, b, s=area, c='b')
    plt.plot(c,d,c='r')
    plt.plot(c,e,c='g')
    plt.title('Pairs of random variables, QLR (red), LR (green)')
    plt.xlabel('x')
    plt.ylabel('y')
               
u = widgets.ToggleButtons(options=['(a)', '(b)'],
    description='Case',
    disabled=False,
    button_style='info', 
    tooltip='Description',
#     icon='check'
) 


u2 = widgets.IntSlider(description='N', min = 10, max = 200, step = 10, value = 40)
z = widgets.interactive(LR, w = u, M = u2) 
display(z)
        

## 7.3 Kalman Filter

You will recall the model described in the book:

\begin{eqnarray*}
X(n+1) &=& AX(n) + V(n) \\
Y(n) &=& CX(n) + W(n).
\end{eqnarray*}

In these expressions, $X$ and $Y$ are vectors and $V, W$ are uncorrelated across time, zero mean. Also,
$V(n)$ and $W(n)$ are uncorrelated and have covariance matrices $\Sigma_V$ and $\Sigma_W$.
We assume that $X(0)$ is zero-mean and has covariance $\Sigma_0$.  

The goal is to calculate $\bar X(n) = L[X(n) | Y^n]$ recursively, where $Y^n = \{Y(0), \ldots, Y(n)\}$.  The Kalman Filter is as follows:

\begin{eqnarray*}
\bar X(n) &=& A \bar X(n-1) + K_n [Y(n) - CA \bar X(n-1)] \tag{7.1}\\
K_n &=& S_nC'[CS_nC' + \Sigma_W]^{-1} \tag{7.2}\\
S_n &=& A \Sigma_{n-1} A' + \Sigma_V \tag{7.3} \\
\Sigma_n &=& (I - K_nC)S_n. \tag{7.4}
\end{eqnarray*}

Moreover, $\Sigma_n = cov(X(n) - \bar X(n))$.

### Examples

In the simulations below, we explore the four examples shown in the book:

**Example 1**:  Random walk with noisy observations.  By choice, we start the walk in $X(0) = 20$ even though it is assume to have zero mean.

\begin{eqnarray*}
X(n+1) &=& X(n) + V(n) \\
Y(n) &=& X(n) + W(n) \\
var(V(n)) &=& varV, var(W(n)) = varW.
\end{eqnarray*}

In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import random
import numpy.linalg
from ipywidgets import *
#from IPython.display import display


def KF1(v,w):  # LR 
    N = 100
    t = np.arange(0,N)
    X = np.arange(0.0,N)
    Xb = np.arange(0.0,N)
    Sigma = v
    S = 0
    K = S/(S + w)
    
    
    Y = np.arange(0.0,N)
    X[0] = 20
    Xb[0] = 0
    Y[0] = X[0] + w*np.random.normal()
    
    
    for k in range(1,N):
        X[k]= X[k-1] + v*np.random.normal()
        Y[k] = X[k] + w*np.random.normal() 
        Xb[k] = Xb[k-1] + K*(Y[k] - Xb[k-1])
        S = Sigma + v
        K = S/(S + w)
        Sigma = (1 - K)*S
    
    plt.figure(figsize = (14,6))
    plX, = plt.plot(t,X,'black')
    plXb, =plt.plot(t,Xb,'r')
    plY, = plt.plot(t,Y,'blue')
    plt.title('States, Observations, Estimates')
    plt.xlabel('n')
    plt.legend([plX, (plX, plXb), (plX, plXb, plY)], ["$X(n)$", "$Xest(n)$","$Y(n)$"])
            




u1 = widgets.IntSlider(description='varV', min = 0, max = 12, step = 0.5, value = 2)
u2 = widgets.IntSlider(description='varW', min = 0, max = 12, step = 0.5, value = 4)
        
z = widgets.interactive(KF1, v = u1, w = u2) 
display(z)
        

**Example 2**:  Random walk with unknown drift

\begin{eqnarray*}
X_1(n+1) &=& X_1(n) + X_2(n) + V(n) \\
X_2(n+1) &=& X_2(n)  \\
Y(n) &=& X_1(n) + W(n) \\
var(V(n)) &=& varV, var(W(n)) = varW.
\end{eqnarray*}

In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import random
import numpy.linalg
import numpy.matlib
from ipywidgets import *
#from IPython.display import display


def KF2(v,w):  # LR 
    N = 30
    t = np.arange(0,N)
    X = np.zeros((2,N))
    Xb = np.zeros((2,N))
    Y = np.arange(0.0,N)
    C = np.array([[1, 0]])
    S = np.array([[v, 0],[0,v]])
    K = np.dot(S,C.transpose())/(S[0,0] + w)
    A = np.array([[1, 1],[0,1]])    
    Sv = np.array([[v, 0],[0,0]])
    Sx = S
    Sw = w
    X[0,0] = 20
    X[1,0] = 1.2
    Xb[0,0] = 0
    Xb[1,0] = 1
    Y[0] = X[0,0] + w*np.random.normal()
        
    for k in range(1,N):
        X[0,k]= (np.dot(A,X[:,k-1]))[0] + v**0.5*np.random.normal()
        X[1,k]= (np.dot(A,X[:,k-1]))[1] 
        Y[k] = X[0,k] + w**0.5*np.random.normal() 
        for i in range(2):
            Xb[i,k] = np.dot(A,Xb[:,k-1])[i] + (K*(Y[k]- Xb[0,k-1]))[i]
        S = np.dot(A,np.dot(Sx, A.T)) + Sv
        K = np.dot(S,C.T)/(S[0,0] + w)
        Sx = np.dot((np.identity(2) - np.dot(K,C)),S)    
    plt.figure(figsize = (14,6))
    plX0, = plt.plot(t,X[0,:],'black',label='$X_1(n)$')
    plt.legend()
    plX1, = plt.plot(t,X[1,:],'green',label='$X_2(n)$')
    plt.legend()
    plXb0, = plt.plot(t,Xb[0,:],'r', label ='$Xe_1(n)$')
    plt.legend()
    plXb1, = plt.plot(t,Xb[1,:],'brown', label ='$Xe_1(n)$')
    plt.legend()
    plY, = plt.plot(t,Y,'blue', label='$Y(n)$')
    plt.legend()
    plt.title('States, Observations, Estimates')
    plt.xlabel('n')
    
u1 = widgets.IntSlider(description='varV', min = 0, max = 12, step = 0.5, value = 2)
u2 = widgets.IntSlider(description='varW', min = 0, max = 12, step = 0.5, value = 4)
        
z = widgets.interactive(KF2, v = u1, w = u2) 
display(z)
        

**Example 3**:  Random walk with changing drift

\begin{eqnarray*}
X_1(n+1) &=& X_1(n) + X_2(n) + V_1(n) \\
X_2(n+1) &=& X_2(n) + V_2(n) \\
Y(n) &=& X_1(n) + W(n) \\
var(V_1(n)) &=& varV1, var(V_2(n)) = varV2, var(W(n)) = varW.
\end{eqnarray*}

In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import random
import numpy.linalg
import numpy.matlib
from ipywidgets import *
#from IPython.display import display


def KF3(v1,v2,w):  # LR 
    N = 20
    t = np.arange(0,N)
    X = np.zeros((2,N))
    Xb = np.zeros((2,N))
    Y = np.arange(0.0,N)
    C = np.array([[1, 0]])
    S = v1*np.identity(2)
    K = np.dot(S,C.transpose())/(S[0,0] + w)
    A = np.array([[1, 1],[0,1]])    
    Sv = np.array([[v1, 0],[0,v2]])
    Sx = Sv
    Sw = w
    X[0,0] = 20
    X[1,0] = 5
    Xb[0,0] = 0
    Xb[1,0] = 1
    Y[0] = X[0,0] + w**0.5*np.random.normal()
        
    for k in range(1,N):
        X[:,k]= np.dot(A,X[:,k-1]) + np.random.normal((2,1))
        Y[k] = X[0,k] + w*np.random.normal() 
        for i in range(2):
            Xb[i,k] = np.dot(A,Xb[:,k-1])[i] + (K*(Y[k]- Xb[0,k-1]))[i]
        S = np.dot(A,np.dot(Sx, A.T)) + Sv
        K = np.dot(S,C.T)/(S[0,0] + w)
        Sx = np.dot((np.identity(2) - np.dot(K,C)),S)    
    plt.figure(figsize = (14,6))
    plX0, = plt.plot(t,X[0,:],'black',label='$X_1(n)$')
    plt.legend()
    plX1, = plt.plot(t,X[1,:],'green',label='$X_2(n)$')
    plt.legend()
    plXb0, = plt.plot(t,Xb[0,:],'r', label ='$Xe_1(n)$')
    plt.legend()
    plXb1, = plt.plot(t,Xb[1,:],'brown', label ='$Xe_1(n)$')
    plt.legend()
    plY, = plt.plot(t,Y,'blue', label='$Y(n)$')
    plt.legend()
    plt.title('States, Observations, Estimates')
    plt.xlabel('n')
    
u11 = widgets.IntSlider(description='varV1', min = 0, max = 12, step = 0.5, value = 2)
u12 = widgets.IntSlider(description='varV2', min = 0, max = 12, step = 0.5, value = 2)
u2 = widgets.IntSlider(description='varW', min = 0, max = 12, step = 0.5, value = 4)
        
z = widgets.interactive(KF3, v1 = u11, v2 = u12, w = u2) 
display(z)
        

**Example 4**:  Falling Object

\begin{eqnarray*}
X_1(n+1) &=& X_1(n) + X_2(n) + V(n) \\
X_2(n+1) &=& X_2(n)  \\
Y(n) &=& X_1(n) + W(n) \\ 
Z(n) &=& X_1(n) - 0.5gn^2 \\
\bar Z(n) &=& \bar X_1(n) - 0.5gn^2 \\
var(V(n)) &=& varV, var(W(n)) = varW.
\end{eqnarray*}

In this expression, $g$ is the gravitation constant normalized for the unit of time.

In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import random
import numpy.linalg
import numpy.matlib
from ipywidgets import *
#from IPython.display import display



def KF4(v,w):  # LR 
    N = 20
    t = np.arange(0,N)
    X = np.zeros((2,N))
    Xb = np.zeros((2,N))
    Y = np.arange(0.0,N)
    Z = np.arange(0.0,N)
    Zb = np.arange(0.0,N)
    C = np.array([[1, 0]])
    S = np.array([[v, 0],[0,v]])
    K = np.dot(S,C.transpose())/(S[0,0] + w)
    A = np.array([[1, 1],[0,1]])    
    Sv = np.array([[v, 0],[0,0]])
    Sx = S
    Sw = w
    X[0,0] = 400
    Z[0] = X[0,0]
    X[1,0] = 1.2
    Xb[0,0] = 410
    Zb[0] = Xb[0,0]
    Xb[1,0] = 1
    g = 2
    Y[0] = X[0,0] + w**0.5*np.random.normal()
        
    for k in range(1,N):
        X[:,k]= np.dot(A,X[:,k-1]) + np.random.normal((2,1))
        Z[k] = X[0,k] - 0.5*g*k**2
        Y[k] = X[0,k] + w*np.random.normal() 
        for i in range(2):
            Xb[i,k] = np.dot(A,Xb[:,k-1])[i] + (K*(Y[k]- Xb[0,k-1]))[i]
        Zb[k] = Xb[0,k]- 0.5*g*k**2
        S = np.dot(A,np.dot(Sx, A.T)) + Sv
        K = np.dot(S,C.T)/(S[0,0] + w)
        Sx = np.dot((np.identity(2) - np.dot(K,C)),S)    
    plt.figure(figsize = (14,6))
    plX0, = plt.plot(t,Z,'black',label='$Z(n)$')
    plt.legend()
    plX1, = plt.plot(t,Zb,'green',label='$Ze(n)$')
    plt.legend()
    plt.title('States, Estimates')
    plt.xlabel('n')
    
u1 = widgets.IntSlider(description='varV', min = 0, max = 12, step = 0.5, value = 2)
u2 = widgets.IntSlider(description='varW', min = 0, max = 12, step = 0.5, value = 2)
z = widgets.interactive(KF4, v = u1, w = u2) 
display(z)
        